In [ ]:
import torch

# ^^^ pyforest auto-imports - don't write above this line
%load_ext autoreload
%autoreload 2

In [ ]:
from copy import deepcopy

from cfg import ModelCfg, TrainCfg
from dataset import CPSC2021
from torch.nn.parallel import DataParallel as DP
from torch.nn.parallel import DistributedDataParallel as DDP
from tqdm.auto import tqdm
from trainer import _MODEL_MAP, CPSC2021Trainer, _set_task

In [ ]:
TrainCfg.db_dir = "/home/wenhao/Jupyter/wenhao/data/CPSC2021/"

In [ ]:
CPSC2021.__DEBUG__ = False

In [ ]:
task = "qrs_detection"

In [ ]:
ds_train = CPSC2021(TrainCfg, training=True, task=task, lazy=False)
ds_val = CPSC2021(TrainCfg, training=False, task=task, lazy=False)

In [ ]:
train_config = deepcopy(TrainCfg)

train_config.main.loss = "AsymmetricLoss"  # "MaskedBCEWithLogitsLoss"
train_config.main.loss_kw = ED(gamma_pos=0, gamma_neg=1, implementation="deep-psp")

# train_config.batch_size = 4
_set_task(task, train_config)
model_config = deepcopy(ModelCfg[task])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model_cls = _MODEL_MAP[train_config[task].model_name]

In [ ]:
model = model_cls(config=model_config)

In [ ]:
model.module_size_

In [ ]:
if task != "rr_lstm" and torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
trainer = CPSC2021Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_val)

In [ ]:
trainer.train()